In [37]:
### Library Imports
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import warnings
warnings.filterwarnings("ignore")


In [38]:
### Import S&P500 Data
def get_data(tickers, start_date, end_date):
    data = pd.DataFrame()
    for ticker in tickers:
        data[ticker] = yf.download(ticker, start_date, end_date)['Adj Close']
    
    return data

### Return Stock Data
tickers = ['^GSPC']  # Ticker symbol for the S&P 500
start_date = '2019-01-01'
end_date = '2024-01-01'
stock_data = get_data(tickers, start_date, end_date)
print(stock_data.tail())

# Get most recent stock price
s_0 = stock_data['^GSPC'].iloc[-1]
print('The most recent stock price is: ', round(s_0, 4))

[*********************100%%**********************]  1 of 1 completed

                  ^GSPC
Date                   
2023-12-22  4754.629883
2023-12-26  4774.750000
2023-12-27  4781.580078
2023-12-28  4783.350098
2023-12-29  4769.830078
The most recent stock price is:  4769.8301


In [39]:
### Set input parameters
# Option parameters
T = 1.0          # Time to maturity (in years)
r = 0.05         # Risk-free rate
n_simulations = 100  # Number of simulations
n_steps = 252    # Number of time steps (252 trading days in a year)
dt = T / n_steps # Time increment

# Determine strike price
k = s_0 * 1.05 # lookback_call or barrier_call (5% increase over current stock price (bullish))
print('The computed option strike price is:', round(k, 4))

# Compute Sigma - stddev * sqrt(252)
daily_rets = stock_data.pct_change().dropna()
#print(daily_rets.tail())
sigma = daily_rets['^GSPC'].std() * np.sqrt(252)
print('The computed sigma is: ', round(sigma, 4))


The computed option strike price is: 5008.3216
The computed sigma is:  0.2132


In [45]:
### Simulate Price Paths
def price_paths(S0, T, r, sigma, dt, n_steps, n_simulations):
    # Initialize the array for storing simulated stock price paths
    S = np.zeros((n_steps + 1, n_simulations))
    S[0] = S0
    # Set the seed for reproducibility
    np.random.seed(42)
    # Simulate price paths
    for t in range(1, n_steps + 1):
        Z = np.random.standard_normal(n_simulations)
        S[t] = S[t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    
    return S

### Display Price Paths
price_paths(s_0, T, r, sigma, dt, n_steps, n_simulations)


array([[4769.83007812, 4769.83007812, 4769.83007812, ..., 4769.83007812,
        4769.83007812, 4769.83007812],
       [4802.28080547, 4761.49477969, 4812.0295101 , ..., 4787.1033202 ,
        4770.67385691, 4755.33808976],
       [4712.35251134, 4735.17906264, 4790.44652734, ..., 4797.51750484,
        4774.92204592, 4683.39412001],
       ...,
       [5471.13762873, 3287.74121318, 5481.30060949, ..., 7412.58327109,
        5067.99353048, 4420.37242654],
       [5484.30304668, 3288.63828982, 5450.23721276, ..., 7593.78728786,
        5009.4969682 , 4387.961909  ],
       [5505.91471601, 3277.7012869 , 5443.50705641, ..., 7586.1239106 ,
        4923.22448021, 4375.63239956]])

In [ ]:
### Plot Price Paths


In [ ]:
### Calculate option Premiums for Lookback and Barrier options

